# Developing a contructive heuristic that has equal pobability of generating every possible feasible solution.

Since each order must be assigned to one of the machines exactly once, it would suffice to produce a random $k$-way partition (allowing empty partitions), from the set of orders, where $k$ is the number of machines.
We were able to find algorithms that produces the number of $k$-way partitions of a set, not allowing empty sets, which is called the Sterling Number of the Second Degree or $S(n,k)$ (see sterling.py).

In [20]:
from sterling import count_part

n = 30
k = 3

print(f"S({n},{k}): {count_part(n,k)}")

S(30,3): 34314651811530


We also found an algorithm that produces the $n$-th $k$-way partition of a set.

In [21]:
from sterling import gen_part

n = 3
k = 2

l = list(range(1, n+1))
for part_i in range(count_part(n, k)):
    print(f'{part_i}: {gen_part(l, k, part_i)}')

0: [[1], [2, 3]]
1: [[1, 2], [3]]
2: [[1, 3], [2]]


The number of $k$-way partitions of a set **allowing empty partitions** $S^*(n,k)$ was found to be $\sum_{l=1}^k S(n,l)$, since with $l-k$ empty partitions, there are $S(n,l)$ ways to partition the set into the $l$ remaining subsets.

In [22]:
def count_part_empty_allowed(n,k):
    return sum([count_part(n, k-e) for e in range(k)])

n = 30
k = 3

print(f"S({n},{k}):  {count_part(n,k)}")
print(f"S*({n},{k}): {count_part_empty_allowed(n,k)}")

S(30,3):  34314651811530
S*(30,3): 34315188682442


Determining the $i^{th}$ $k$-way partition allowing empty sets was a bit tricky. Generating all $S^*(n,k)$ empty-allowed $k$-way paritions could be done like so:

In [23]:
n = 4
k = 3
l = list(range(1, n+1))

part_counts = [(m, count_part(n, m)) for m in range(1, k+1)]

print(f"m  S(n,m)")
for m, part_count in part_counts:
    print(f'{m}: {part_count}')

print(f"\n m  i")
for m, part_count in part_counts:
    for part_i in range(part_count):
        part = [[] for _ in range(k-m)] + gen_part(l, m, part_i)
        print(f'{m, part_i}: {part}')

m  S(n,m)
1: 1
2: 7
3: 6

 m  i
(1, 0): [[], [], [1, 2, 3, 4]]
(2, 0): [[], [1], [2, 3, 4]]
(2, 1): [[], [1, 2], [3, 4]]
(2, 2): [[], [1, 3], [2, 4]]
(2, 3): [[], [1, 4], [2, 3]]
(2, 4): [[], [1, 2, 3], [4]]
(2, 5): [[], [1, 2, 4], [3]]
(2, 6): [[], [1, 3, 4], [2]]
(3, 0): [[1], [2], [3, 4]]
(3, 1): [[1], [2, 3], [4]]
(3, 2): [[1], [2, 4], [3]]
(3, 3): [[1, 2], [3], [4]]
(3, 4): [[1, 3], [2], [4]]
(3, 5): [[1, 4], [2], [3]]


We then adapted this algorithm to generate the $i^{th}$ $k$-way partition allowing empty sets like so:

In [24]:
def get_ith_part_allow_empty(A,k,i):
    
    n = len(A)
    
    # Deteremine m
    skipped_parts = 0
    for e in range(k):
        part_count = count_part(n, k-e)
        
        # The index i given k >= m
        i_given_e = i - skipped_parts
        
        # Skip if i_given_m is larger than S(n,m)
        if (i_given_e) >= part_count:
            skipped_parts += part_count
            continue
        
        # Return k-m empty sets + i_given_m'th m-way partition.
        return [[] for _ in range(e)] + gen_part(A, k-e, i_given_e)


# Print the partitions
n = 4
k = 3
A = list(range(1, n+1))

for part_i in range(count_part_empty_allowed(n, k)):
    print(f'{part_i}: {get_ith_part_allow_empty(A,k,part_i)}')

0: [[1], [2], [3, 4]]
1: [[1], [2, 3], [4]]
2: [[1], [2, 4], [3]]
3: [[1, 2], [3], [4]]
4: [[1, 3], [2], [4]]
5: [[1, 4], [2], [3]]
6: [[], [1], [2, 3, 4]]
7: [[], [1, 2], [3, 4]]
8: [[], [1, 3], [2, 4]]
9: [[], [1, 4], [2, 3]]
10: [[], [1, 2, 3], [4]]
11: [[], [1, 2, 4], [3]]
12: [[], [1, 3, 4], [2]]
13: [[], [], [1, 2, 3, 4]]


Now, all thats left is to permute the resulting partitions:

In [25]:
from itertools import permutations

n = 4
k = 3
A = list(range(1, n+1))

for part_i in range(count_part_empty_allowed(n, k)):
    part = get_ith_part_allow_empty(A,k,part_i)
    for pi, perm in enumerate(permutations(part, k)):
        print(f'{part_i, pi}: {perm}')
    print()

(0, 0): ([1], [2], [3, 4])
(0, 1): ([1], [3, 4], [2])
(0, 2): ([2], [1], [3, 4])
(0, 3): ([2], [3, 4], [1])
(0, 4): ([3, 4], [1], [2])
(0, 5): ([3, 4], [2], [1])

(1, 0): ([1], [2, 3], [4])
(1, 1): ([1], [4], [2, 3])
(1, 2): ([2, 3], [1], [4])
(1, 3): ([2, 3], [4], [1])
(1, 4): ([4], [1], [2, 3])
(1, 5): ([4], [2, 3], [1])

(2, 0): ([1], [2, 4], [3])
(2, 1): ([1], [3], [2, 4])
(2, 2): ([2, 4], [1], [3])
(2, 3): ([2, 4], [3], [1])
(2, 4): ([3], [1], [2, 4])
(2, 5): ([3], [2, 4], [1])

(3, 0): ([1, 2], [3], [4])
(3, 1): ([1, 2], [4], [3])
(3, 2): ([3], [1, 2], [4])
(3, 3): ([3], [4], [1, 2])
(3, 4): ([4], [1, 2], [3])
(3, 5): ([4], [3], [1, 2])

(4, 0): ([1, 3], [2], [4])
(4, 1): ([1, 3], [4], [2])
(4, 2): ([2], [1, 3], [4])
(4, 3): ([2], [4], [1, 3])
(4, 4): ([4], [1, 3], [2])
(4, 5): ([4], [2], [1, 3])

(5, 0): ([1, 4], [2], [3])
(5, 1): ([1, 4], [3], [2])
(5, 2): ([2], [1, 4], [3])
(5, 3): ([2], [3], [1, 4])
(5, 4): ([3], [1, 4], [2])
(5, 5): ([3], [2], [1, 4])

(6, 0): ([], [1], [2, 

In [26]:
partition = [[], [], [1]]
all_perms = list(permutations(partition))
for perm in all_perms:
    print(perm)

([], [], [1])
([], [1], [])
([], [], [1])
([], [1], [])
([1], [], [])
([1], [], [])


One small problem: As you can see, permutations is retuning duplicates:

In [27]:
all_perms[0] == all_perms[2]

True

Turns out we actually want the distinct permutations:

In [28]:
from more_itertools import distinct_permutations

for perm in distinct_permutations(partition):
    print(perm)

([], [], [1])
([], [1], [])
([1], [], [])


Now we have to determine the amount of distict permutations. It will be dependent on the amount of empty partitions $e$. According to (this)[https://math.stackexchange.com/questions/1759845/how-do-i-calculate-the-number-of-unique-permutations-in-a-list-with-repeated-ele] stackexchange answer, it should be $\frac {k!} {\sum_{i \in I} i!}$ where $k$ is the number of elements in the list and $I$ is a list of counts of duplicates. In our case that would be $\frac {k!} {e!}$ where $k$ is the size of the partition and $e$ the number of empty partitions. Lets check this for a bunch of partitions

In [29]:
from math import factorial


n = 6
k = 5
A = list(range(n))

print("(num_of_empty_parts, distinct_partitions == est_distinct_partitions)")
for part_i in range(count_part_empty_allowed(n, k)):
    part = get_ith_part_allow_empty(A,k,part_i)
    all_perms = list(distinct_permutations(part, k))
    e = len([el for el in part if el == []])
    print(f"{e}: \t{len(all_perms)} == {factorial(k) // factorial(e)}")

(num_of_empty_parts, distinct_partitions == est_distinct_partitions)
0: 	120 == 120
0: 	120 == 120
0: 	120 == 120
0: 	120 == 120
0: 	120 == 120
0: 	120 == 120
0: 	120 == 120
0: 	120 == 120
0: 	120 == 120
0: 	120 == 120
0: 	120 == 120
0: 	120 == 120
0: 	120 == 120
0: 	120 == 120
0: 	120 == 120
1: 	120 == 120
1: 	120 == 120
1: 	120 == 120
1: 	120 == 120
1: 	120 == 120
1: 	120 == 120
1: 	120 == 120
1: 	120 == 120
1: 	120 == 120
1: 	120 == 120
1: 	120 == 120
1: 	120 == 120
1: 	120 == 120
1: 	120 == 120
1: 	120 == 120
1: 	120 == 120
1: 	120 == 120
1: 	120 == 120
1: 	120 == 120
1: 	120 == 120
1: 	120 == 120
1: 	120 == 120
1: 	120 == 120
1: 	120 == 120
1: 	120 == 120
1: 	120 == 120
1: 	120 == 120
1: 	120 == 120
1: 	120 == 120
1: 	120 == 120
1: 	120 == 120
1: 	120 == 120
1: 	120 == 120
1: 	120 == 120
1: 	120 == 120
1: 	120 == 120
1: 	120 == 120
1: 	120 == 120
1: 	120 == 120
1: 	120 == 120
1: 	120 == 120
1: 	120 == 120
1: 	120 == 120
1: 	120 == 120
1: 	120 == 120
1: 	120 == 120
1: 	120 == 120
1

Lets make a function get the number of distinct permutations given a count of the number of duplicates for a single value - in our case, the empty set.

In [30]:
def get_distinct_permutation_count(n, n_dupes):
    return factorial(n) // factorial(n_dupes)

Now lets design a function to get the number of the sum of the number of distinct permutations for all empty-allowed k-way partitions of a list:

In [31]:

# [n,k]: [(Colulative Number of Solutions, Number of empty Partitions, Number of non-empty Partitions, Number of permutations)]
cumulative_solution_count_cache = {}
def get_numberof_distperm_kwayparts(n, k, verbose = 0):
    
    cumulative_count = 0
    
    # Initialize cache entry    
    key = n,k
    cumulative_solution_count_cache[key] = []

    
    if verbose > 0:
        print("empty   partitions   permutations   total")
        
    n_distperms = []
    
    # For each (number of empty partitions)
    for e in range(0,k):
        
        # Number of ways to make the remaining k-e partitions.
        n_parts = count_part(n, k-e)
        
        # Number of ways to permute a partition with e duplicates
        n_perms = get_distinct_permutation_count(k, e)
        
        # Add values multiplied
        n_distperms += [n_parts * n_perms]
        
        # Add to cache
        cumulative_count += n_parts * n_perms
        cumulative_solution_count_cache[key] += [(cumulative_count, e, n_parts, n_perms)]
        
        if verbose > 0:
            print(f"{e:<7} {n_parts:<12} {n_perms:<14} {n_parts * n_perms}")
    
    return sum(n_distperms)


get_numberof_distperm_kwayparts(5, 3, verbose = 1)

empty   partitions   permutations   total
0       25           6              150
1       15           6              90
2       1            3              3


243

And now, one that lists us all of the distinct permutations of all possible $k-way$ partitions, allowing empty partitions, of a set. We will call the result the set of all solutions for A.

In [32]:
def get_distperms_kwayparts(A, k, verbose = 0):
    
    distperms = []
    n = len(A)
    
    # For each (number of empty partitions)
    for e in range(0,k):
        
        # Empty parts
        e_parts: list[list[int]] = [[] for _ in range(e)]
        
        # Get all partitions with e empty sets
        n_parts = count_part(n, k-e)
        parts = []
        for part_i in range(n_parts):
            parts += [[*e_parts, *gen_part(A, k-e, part_i)]]
        
        
        if verbose > 0:
            print(f"{e} empty parts: ({n_parts} partitions, {get_distinct_permutation_count(k, e)} permutations each)")
        
        # Get all distinct permutations
        all_perms = []
        for part in parts:
            perms = list(distinct_permutations(part))
            all_perms += perms
            # print(f"\n{part}: ({len(perms)})")
            print()
            for perm in perms:
                print(perm)
        
        distperms += all_perms
        
        print()
        print()
        
    return distperms

#
distpermparts = get_distperms_kwayparts(list(range(1, 4)), 3, verbose = 1)

0 empty parts: (1 partitions, 6 permutations each)

([1], [2], [3])
([1], [3], [2])
([2], [1], [3])
([2], [3], [1])
([3], [1], [2])
([3], [2], [1])


1 empty parts: (3 partitions, 6 permutations each)

([], [1], [2, 3])
([], [2, 3], [1])
([1], [], [2, 3])
([1], [2, 3], [])
([2, 3], [], [1])
([2, 3], [1], [])

([], [1, 2], [3])
([], [3], [1, 2])
([1, 2], [], [3])
([1, 2], [3], [])
([3], [], [1, 2])
([3], [1, 2], [])

([], [1, 3], [2])
([], [2], [1, 3])
([1, 3], [], [2])
([1, 3], [2], [])
([2], [], [1, 3])
([2], [1, 3], [])


2 empty parts: (1 partitions, 3 permutations each)

([], [], [1, 2, 3])
([], [1, 2, 3], [])
([1, 2, 3], [], [])




Now a function to give us the $i^{th}$ solution for a given set $A$, partition count $k$.

In [33]:
def get_ith_solution(A, k, i):
    
    n = len(A)
    key = n,k
    if key not in cumulative_solution_count_cache:
        n_solutions = get_numberof_distperm_kwayparts(n,k)
                
    cache = cumulative_solution_count_cache[n, k]
    
    #DEBUG
    
    last_s_count_cum = 0
    for s_count_cum, e, n_parts, n_perms in cache:
        
        # Skip if higher
        if i >= s_count_cum:
            last_s_count_cum = s_count_cum
            continue
        
        # Get local solution index
        i_given_e = i - last_s_count_cum
        
        i_part, i_perm  = divmod(i_given_e, n_perms)
                
        # Empty parts
        e_parts: list[list[int]] = [[] for _ in range(e)]
        
        
        part = [*e_parts, *gen_part(A, k-e, i_part)]
        
        # Get i_perm'th partition
        # print(f"Getting permutation #{i_perm} of partition #{i_part} granted {e} empties.")
        perm = list(distinct_permutations(part))[i_perm]

        return perm

Lets list all solutions where $n = 5$, $k = 3$

In [34]:
n = 5
k = 3
A = list(range(5))
n_solutions = get_numberof_distperm_kwayparts(n, k)

# print(n_solutions)
for i in range(n_solutions):
    print(f"{i}: {get_ith_solution(A, k, i)}")

0: ([0], [1], [2, 3, 4])
1: ([0], [2, 3, 4], [1])
2: ([1], [0], [2, 3, 4])
3: ([1], [2, 3, 4], [0])
4: ([2, 3, 4], [0], [1])
5: ([2, 3, 4], [1], [0])
6: ([0], [1, 2], [3, 4])
7: ([0], [3, 4], [1, 2])
8: ([1, 2], [0], [3, 4])
9: ([1, 2], [3, 4], [0])
10: ([3, 4], [0], [1, 2])
11: ([3, 4], [1, 2], [0])
12: ([0], [1, 3], [2, 4])
13: ([0], [2, 4], [1, 3])
14: ([1, 3], [0], [2, 4])
15: ([1, 3], [2, 4], [0])
16: ([2, 4], [0], [1, 3])
17: ([2, 4], [1, 3], [0])
18: ([0], [1, 4], [2, 3])
19: ([0], [2, 3], [1, 4])
20: ([1, 4], [0], [2, 3])
21: ([1, 4], [2, 3], [0])
22: ([2, 3], [0], [1, 4])
23: ([2, 3], [1, 4], [0])
24: ([0], [1, 2, 3], [4])
25: ([0], [4], [1, 2, 3])
26: ([1, 2, 3], [0], [4])
27: ([1, 2, 3], [4], [0])
28: ([4], [0], [1, 2, 3])
29: ([4], [1, 2, 3], [0])
30: ([0], [1, 2, 4], [3])
31: ([0], [3], [1, 2, 4])
32: ([1, 2, 4], [0], [3])
33: ([1, 2, 4], [3], [0])
34: ([3], [0], [1, 2, 4])
35: ([3], [1, 2, 4], [0])
36: ([0], [1, 3, 4], [2])
37: ([0], [2], [1, 3, 4])
38: ([1, 3, 4], [0], [

Here they are segmented (used for debugging)

In [35]:
i = 0
for c in cumulative_solution_count_cache[n,k]:
    print('\n')
    print(c)
    for j in range(i, c[0]):
        if (j-i)%c[3] == 0:
            print()
        print(f"{j}: {get_ith_solution(A, k, j)}")
    i = c[0]



(150, 0, 25, 6)

0: ([0], [1], [2, 3, 4])
1: ([0], [2, 3, 4], [1])
2: ([1], [0], [2, 3, 4])
3: ([1], [2, 3, 4], [0])
4: ([2, 3, 4], [0], [1])
5: ([2, 3, 4], [1], [0])

6: ([0], [1, 2], [3, 4])
7: ([0], [3, 4], [1, 2])
8: ([1, 2], [0], [3, 4])
9: ([1, 2], [3, 4], [0])
10: ([3, 4], [0], [1, 2])
11: ([3, 4], [1, 2], [0])

12: ([0], [1, 3], [2, 4])
13: ([0], [2, 4], [1, 3])
14: ([1, 3], [0], [2, 4])
15: ([1, 3], [2, 4], [0])
16: ([2, 4], [0], [1, 3])
17: ([2, 4], [1, 3], [0])

18: ([0], [1, 4], [2, 3])
19: ([0], [2, 3], [1, 4])
20: ([1, 4], [0], [2, 3])
21: ([1, 4], [2, 3], [0])
22: ([2, 3], [0], [1, 4])
23: ([2, 3], [1, 4], [0])

24: ([0], [1, 2, 3], [4])
25: ([0], [4], [1, 2, 3])
26: ([1, 2, 3], [0], [4])
27: ([1, 2, 3], [4], [0])
28: ([4], [0], [1, 2, 3])
29: ([4], [1, 2, 3], [0])

30: ([0], [1, 2, 4], [3])
31: ([0], [3], [1, 2, 4])
32: ([1, 2, 4], [0], [3])
33: ([1, 2, 4], [3], [0])
34: ([3], [0], [1, 2, 4])
35: ([3], [1, 2, 4], [0])

36: ([0], [1, 3, 4], [2])
37: ([0], [2], [1, 3, 4

Now lets see if we can generate a random solution for **big** problem

In [36]:
n = 30
k = 3
A = list(range(n))
n_solutions = get_numberof_distperm_kwayparts(n, k)
print(n_solutions)

205891132094649


In [37]:
from random import randint


i = randint(0, n_solutions)

print(f"{i}: {get_ith_solution(A, k, i)}")

85028669230379: ([8, 9, 10, 11, 18, 22, 27, 28, 29], [3, 4, 7, 12, 13, 16, 17, 20, 21, 25, 26], [0, 1, 2, 5, 6, 14, 15, 19, 23, 24])


Now lets use set notation to describe the entire solution space.

In [38]:
for i in range(5):
    print(f"{get_ith_solution(A, k, i)}")
print("...")
for i in range(n_solutions-5, n_solutions):
    print(f"{get_ith_solution(A, k, i)}")

([0], [1], [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])
([0], [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29], [1])
([1], [0], [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])
([1], [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29], [0])
([2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29], [0], [1])
...
([1], [], [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])
([1], [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29], [])
([], [], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])
([], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 

This code is adapted into a [library file](solution_space.py) and the [constructive_heuristics](heuristics_constructive.py) Random class to generate a random solution. This class was then used to make estimations about the solution space in [a notebook detailing exploration of the solution space](test_solution_space.ipynb).